# WESAD Data Cleaning & Exploratory Data Analysis

This notebook combines:
- **Part 1 – Data Cleaning**: WESAD chest sensor data (ACC, ECG, EDA, EMG, Resp, Temp, events, questionnaires)
- **Part 2 – EDA**: Aligned wrist BVP/EDA data at 64 Hz with stress labels (0=baseline, 1=stress, 2=amusement, -1=other)

In [ ]:
! pip install pandas numpy matplotlib seaborn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

%matplotlib inline
plt.style.use("ggplot")

---
## Part 1: Data Cleaning (WESAD Chest Data)
---

In [ ]:
import pandas as pd
import os

merged_dir = '/Users/mostafazamaniturk/Documents/USD/AAI-530/work/WESAD_data/merged_cleaned_data'

# Large files (~55M rows); sample to avoid memory issues
large_files = {'merged_chest_acc.csv', 'merged_wrist_acc.csv'}
large_nrows = 100_000

csv_files = [
    'merged_chest_acc.csv',
    'merged_chest_ecg.csv',
    'merged_chest_eda.csv',
    'merged_chest_emg.csv',
    'merged_chest_resp.csv',
    'merged_chest_temp.csv',
    'merged_event_timings.csv',
    'merged_questionnaire_responses.csv',
    'merged_wrist_acc.csv',
    'merged_wrist_bvp.csv',
    'merged_wrist_eda.csv',
    'merged_wrist_temp.csv',
]

for fname in csv_files:
    path = os.path.join(merged_dir, fname)
    if not os.path.exists(path):
        print(f"File not found: {fname}")
        continue

    nrows = large_nrows if fname in large_files else None
    df = pd.read_csv(path, nrows=nrows)

    print("=" * 80)
    print(f"FILE: {fname}")
    print("=" * 80)
    print("\n.head():\n", df.head())
    print("\n.tail():\n", df.tail())
    print("\n.describe():\n", df.describe(include='all'))
    print("\n")

In [ ]:
! pip install ucimlrepo

In [ ]:
print('Listing contents of MyDrive:')
! ls "/Users/mostafazamaniturk/Documents/USD/AAI-530/work"

If your `WESAD.zip` file is inside a subfolder, for example, named `MyProject`, you would then run:

```python
!ls "/Users/mostafazamaniturk/Documents/USD/AAI-530/work"
```

Once you find the file, use the full path in your code.

In [ ]:
import os
import zipfile

# Define paths using your local structure
base_path = r'/Users/mostafazamaniturk/Documents/USD/AAI-530/work'
zip_path = os.path.join(base_path, 'WESAD.zip')
extract_path = os.path.join(base_path, 'WESAD_data')

if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"Extraction complete! Files are in: {extract_path}")
else:
    print("Zip file not found. Check the path again.")

# Task
Explore the structure of the unzipped WESAD data by listing the contents of the `/content/WESAD_data` directory to identify the main data files.

## Explore Unzipped Data

### Subtask:
Explore the structure of the unzipped WESAD data by listing the contents of the `/content/WESAD_data` directory to identify the main data files.


**Reasoning**:
To explore the structure of the unzipped WESAD data, I will list the contents of the `/content/WESAD_data` directory using the `!ls -F` command as instructed.



In [ ]:
print('Listing contents of the unzipped WESAD data directory:')
!ls -F /Users/mostafazamaniturk/Documents/USD/AAI-530/work/WESAD_data

**Reasoning**:
Since the previous `ls` command revealed a subdirectory named `WESAD/`, I will now list the contents of this subdirectory to further explore the data structure and locate the actual data files.



In [ ]:
print('Listing contents of the WESAD subdirectory:')
!ls -F /Users/mostafazamaniturk/Documents/USD/AAI-530/work/WESAD_data/WESAD

**Reasoning**:
The previous command revealed subject-specific subdirectories (S2/, S3/, etc.) and a `wesad_readme.pdf` file. To understand the data structure further, I will list the contents of one of these subject directories, specifically `S2/`, to see what kind of data files are stored there.



In [ ]:
print('Listing contents of the S2 subdirectory:')
!ls -F /Users/mostafazamaniturk/Documents/USD/AAI-530/work/WESAD_data/WESAD/S2

## Load Data

### Subtask:
Load the relevant WESAD data files into pandas DataFrames for cleaning.


In [ ]:
import pandas as pd
import numpy as np

# Define the base path to the S2 subject data directory
s2_data_path = '/Users/mostafazamaniturk/Documents/USD/AAI-530/work/WESAD_data/WESAD/S2'

# Load the S2.pkl file into a dictionary first
s2_raw_data_dict = pd.read_pickle(f'{s2_data_path}/S2.pkl')

# Initialize dictionaries to hold individual sensor DataFrames
chest_dataframes = {}
wrist_dataframes = {}

# Process 'chest' signals
chest_signals = s2_raw_data_dict['signal']['chest']
for sensor_name, data_array in chest_signals.items():
    if sensor_name == 'ACC':
        # Accelerometer data is 3-axis, create three columns
        chest_dataframes[sensor_name] = pd.DataFrame(data_array, columns=['ACC_x', 'ACC_y', 'ACC_z'])
    else:
        # Other sensors are typically 1D or 2D with one column, flatten and create a DataFrame
        chest_dataframes[sensor_name] = pd.DataFrame(data_array.flatten(), columns=[sensor_name])

# Process 'wrist' signals
wrist_signals = s2_raw_data_dict['signal']['wrist']
for sensor_name, data_array in wrist_signals.items():
    if sensor_name == 'ACC':
        # Accelerometer data is 3-axis, create three columns
        wrist_dataframes[sensor_name] = pd.DataFrame(data_array, columns=['ACC_x', 'ACC_y', 'ACC_z'])
    else:
        # Other sensors are typically 1D or 2D with one column, flatten and create a DataFrame
        wrist_dataframes[sensor_name] = pd.DataFrame(data_array.flatten(), columns=[sensor_name])

# Load the S2_quest.csv file into a pandas DataFrame
df_s2_quest = pd.read_csv(f'{s2_data_path}/S2_quest.csv')

print("Chest DataFrames (first 5 rows of each):")
for sensor_name, df in chest_dataframes.items():
    print(f"\n{sensor_name}:")
    print(df.head())

print("\nWrist DataFrames (first 5 rows of each):")
for sensor_name, df in wrist_dataframes.items():
    print(f"\n{sensor_name}:")
    print(df.head())

print("\ndf_s2_quest head:")
print(df_s2_quest.head())

## Initial Data Inspection

### Subtask:
Perform an initial inspection of the loaded data, including checking the first few rows (.head()), data types (.info()), and descriptive statistics (.describe()) to identify potential issues.


**Reasoning**:
To perform an initial inspection, I will iterate through the `chest_dataframes` and `wrist_dataframes` dictionaries, applying `.head()`, `.info()`, and `.describe()` to each sensor DataFrame. I will also apply these methods to the `df_s2_quest` DataFrame.



In [ ]:
print("--- Initial Data Inspection for Chest Sensor DataFrames ---")
for sensor_name, df in chest_dataframes.items():
    print(f"\n----- {sensor_name} Data (Chest) -----")
    print(f"\n{sensor_name}.head():")
    print(df.head())
    print(f"\n{sensor_name}.info():")
    df.info()
    print(f"\n{sensor_name}.describe():")
    print(df.describe())

print("\n--- Initial Data Inspection for Wrist Sensor DataFrames ---")
for sensor_name, df in wrist_dataframes.items():
    print(f"\n----- {sensor_name} Data (Wrist) -----")
    print(f"\n{sensor_name}.head():")
    print(df.head())
    print(f"\n{sensor_name}.info():")
    df.info()
    print(f"\n{sensor_name}.describe():")
    print(df.describe())

print("\n--- Initial Data Inspection for Questionnaire Data (df_s2_quest) ---")
print("\ndf_s2_quest.head():")
print(df_s2_quest.head())
print("\ndf_s2_quest.info():")
df_s2_quest.info()
print("\ndf_s2_quest.describe():")
print(df_s2_quest.describe())

In [ ]:
import os
import pandas as pd
import numpy as np

# Define paths - works for both local and Colab
base_path = r'/Users/mostafazamaniturk/Documents/USD/AAI-530/work'
wesad_base = os.path.join(base_path, 'WESAD_data', 'WESAD')

# All subject folders to process
SUBJECTS = ['S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S13', 'S14', 'S15', 'S16', 'S17']

def clean_split_list(s):
    parts = s.replace('#', '').split(';')
    return [p.strip() for p in parts if p.strip()]

def process_subject(subject):
    """Load, clean, parse, and save data for one subject."""
    subject_path = os.path.join(wesad_base, subject)
    pkl_path = os.path.join(subject_path, f'{subject}.pkl')
    quest_path = os.path.join(subject_path, f'{subject}_quest.csv')
    
    if not os.path.exists(pkl_path) or not os.path.exists(quest_path):
        print(f"Skipping {subject}: missing pkl or quest file")
        return None
    
    # Load pkl
    raw_data = pd.read_pickle(pkl_path)
    chest_dataframes = {}
    wrist_dataframes = {}
    
    # Process chest signals
    for sensor_name, data_array in raw_data['signal']['chest'].items():
        if sensor_name == 'ACC':
            chest_dataframes[sensor_name] = pd.DataFrame(data_array, columns=['ACC_x', 'ACC_y', 'ACC_z'])
        else:
            chest_dataframes[sensor_name] = pd.DataFrame(data_array.flatten(), columns=[sensor_name])
    
    # Process wrist signals
    for sensor_name, data_array in raw_data['signal']['wrist'].items():
        if sensor_name == 'ACC':
            wrist_dataframes[sensor_name] = pd.DataFrame(data_array, columns=['ACC_x', 'ACC_y', 'ACC_z'])
        else:
            wrist_dataframes[sensor_name] = pd.DataFrame(data_array.flatten(), columns=[sensor_name])
    
    # FULL SESSION: Do NOT truncate. Export entire recording (~1.5 hours per subject).
    for sensor_name in chest_dataframes:
        chest_dataframes[sensor_name] = chest_dataframes[sensor_name].reset_index(drop=True)
    for sensor_name in wrist_dataframes:
        wrist_dataframes[sensor_name] = wrist_dataframes[sensor_name].reset_index(drop=True)
    # Sanity: expect ~90 min per subject (wrist BVP at 64 Hz)
    if 'BVP' in wrist_dataframes:
        dur_min = len(wrist_dataframes['BVP']) / 64.0 / 60.0
        print(f"  {subject}: {len(wrist_dataframes['BVP'])} BVP samples = {dur_min:.1f} min")
    
    # Load and parse questionnaire
    df_quest = pd.read_csv(quest_path)
    order_str = df_quest.iloc[0, 0]
    start_str = df_quest.iloc[1, 0]
    end_str = df_quest.iloc[2, 0]
    
    event_names = clean_split_list(order_str)[1:]
    start_times = {event: float(clean_split_list(start_str)[i+1]) if (i+1) < len(clean_split_list(start_str)) else None 
                   for i, event in enumerate(event_names)}
    end_times = {event: float(clean_split_list(end_str)[i+1]) if (i+1) < len(clean_split_list(end_str)) else None 
                 for i, event in enumerate(event_names)}
    
    df_event_timings = pd.DataFrame({
        'Event': event_names,
        'Start_Time': [start_times.get(e) for e in event_names],
        'End_Time': [end_times.get(e) for e in event_names]
    })
    df_event_timings['Duration'] = df_event_timings['End_Time'] - df_event_timings['Start_Time']
    
    # Parse questionnaire responses
    panas_data, stai_data, dim_data, sssq_data = [], [], [], []
    for i in range(4, len(df_quest)):
        row_string = df_quest.iloc[i, 0]
        if row_string.startswith('# PANAS'):
            parts = row_string.replace('# PANAS;', '').split(';')
            panas_data.append([float(p.strip()) for p in parts if p.strip()])
        elif row_string.startswith('# STAI'):
            parts = row_string.replace('# STAI;', '').split(';')
            stai_data.append([float(p.strip()) for p in parts if p.strip()])
        elif row_string.startswith('# DIM'):
            parts = row_string.replace('# DIM;', '').split(';')
            dim_data.append([float(p.strip()) for p in parts if p.strip()])
        elif row_string.startswith('# SSSQ'):
            parts = row_string.replace('# SSSQ;', '').split(';')
            sssq_data.append([float(p.strip()) for p in parts if p.strip()])
    
    df_panas = pd.DataFrame(panas_data)
    df_stai = pd.DataFrame(stai_data)
    df_dim = pd.DataFrame(dim_data)
    df_sssq = pd.DataFrame(sssq_data)
    
    df_panas.columns = [f'PANAS_{i}' for i in range(len(df_panas.columns))]
    df_stai.columns = [f'STAI_{i}' for i in range(len(df_stai.columns))]
    df_dim.columns = [f'DIM_{i}' for i in range(len(df_dim.columns))]
    df_sssq.columns = [f'SSSQ_{i}' for i in range(len(df_sssq.columns))]
    
    df_combined = pd.concat([df_panas, df_stai, df_dim], axis=1)
    df_sssq_ext = pd.DataFrame(np.nan, index=range(len(df_combined)), columns=df_sssq.columns)
    if not df_sssq.empty:
        df_sssq_ext.iloc[0] = df_sssq.iloc[0]
    df_questionnaire_responses = pd.concat([df_combined, df_sssq_ext], axis=1)
    
    # Save to subject's cleaned_data folder
    cleaned_data_dir = os.path.join(subject_path, 'cleaned_data')
    os.makedirs(cleaned_data_dir, exist_ok=True)
    
    df_event_timings.to_csv(os.path.join(cleaned_data_dir, 'df_event_timings.csv'), index=False)
    df_questionnaire_responses.to_csv(os.path.join(cleaned_data_dir, 'df_questionnaire_responses.csv'), index=False)
    for sensor_name, df in chest_dataframes.items():
        df.to_csv(os.path.join(cleaned_data_dir, f'chest_{sensor_name.lower()}.csv'), index=False)
    for sensor_name, df in wrist_dataframes.items():
        df.to_csv(os.path.join(cleaned_data_dir, f'wrist_{sensor_name.lower()}.csv'), index=False)
    
    return {
        'subject': subject,
        'df_event_timings': df_event_timings,
        'df_questionnaire_responses': df_questionnaire_responses,
        'chest_dataframes': chest_dataframes,
        'wrist_dataframes': wrist_dataframes,
    }

# Process all subjects
all_results = []
for subject in SUBJECTS:
    print(f"Processing {subject}...")
    result = process_subject(subject)
    if result is not None:
        all_results.append(result)
        print(f"  Saved cleaned data for {subject}")

# Merge event timings and questionnaire responses
merged_event_timings = []
merged_questionnaire = []
for r in all_results:
    et = r['df_event_timings'].copy()
    et['subject'] = r['subject']
    merged_event_timings.append(et)
    qr = r['df_questionnaire_responses'].copy()
    qr['subject'] = r['subject']
    merged_questionnaire.append(qr)

df_merged_event_timings = pd.concat(merged_event_timings, ignore_index=True)
df_merged_questionnaire = pd.concat(merged_questionnaire, ignore_index=True)

# Merge sensor data (with subject column)
# Map file names to actual dict keys (chest: Temp/Resp, wrist: TEMP)
sensor_map = {
    'chest_acc': ('chest', 'ACC'), 'chest_ecg': ('chest', 'ECG'), 'chest_emg': ('chest', 'EMG'),
    'chest_eda': ('chest', 'EDA'), 'chest_temp': ('chest', 'Temp'), 'chest_resp': ('chest', 'Resp'),
    'wrist_acc': ('wrist', 'ACC'), 'wrist_bvp': ('wrist', 'BVP'), 'wrist_eda': ('wrist', 'EDA'),
    'wrist_temp': ('wrist', 'TEMP')
}
merged_sensors = {}
for fname, (sensor_type, sensor_key) in sensor_map.items():
    parts = []
    for r in all_results:
        dfs = r['chest_dataframes'] if sensor_type == 'chest' else r['wrist_dataframes']
        if sensor_key in dfs:
            df = dfs[sensor_key].copy()
            df['subject'] = r['subject']
            parts.append(df)
    if parts:
        merged_sensors[fname] = pd.concat(parts, ignore_index=True)

# Save merged dataset
merged_dir = os.path.join(base_path, 'WESAD_data', 'merged_cleaned_data')
os.makedirs(merged_dir, exist_ok=True)
print(f"\nSaving merged dataset to: {merged_dir}")

df_merged_event_timings.to_csv(os.path.join(merged_dir, 'merged_event_timings.csv'), index=False)
df_merged_questionnaire.to_csv(os.path.join(merged_dir, 'merged_questionnaire_responses.csv'), index=False)
for fname, df in merged_sensors.items():
    df.to_csv(os.path.join(merged_dir, f'merged_{fname}.csv'), index=False)

print(f"Saved merged_event_timings.csv ({len(df_merged_event_timings)} rows)")
print(f"Saved merged_questionnaire_responses.csv ({len(df_merged_questionnaire)} rows)")
for fname, df in merged_sensors.items():
    print(f"Saved merged_{fname}.csv ({len(df)} rows)")
print("\nAll cleaned data saved per subject and merged dataset saved successfully!")

## Full-Session Export Pipeline (REVISED)

**Requirements:**
1. **Load from original WESAD pickle files** – Export the entire session (~1.5 hours per subject), not truncated.
2. **Use merged_event_timings.csv** – Map timestamps to true events from the questionnaire-derived timing file.
3. **Label column** – Generate labels: 0=Baseline, 1=Stress (TSST), 2=Amusement (Fun), -1=Other/Transition (Medi, Read).

Event times in `merged_event_timings.csv` (from `*_quest.csv`) are in **minutes** from session start. We convert to seconds when mapping to sensor `time_sec`.

Now, We are going to release information from merged datasets

In [ ]:
import os
import pandas as pd
import numpy as np

merged_dir = '/Users/mostafazamaniturk/Documents/USD/AAI-530/work/WESAD_data/merged_cleaned_data'

# 1. File inventory and sizes
print("=" * 60)
print("MERGED CSV FILES - INVENTORY")
print("=" * 60)
csv_files = sorted([f for f in os.listdir(merged_dir) if f.endswith('.csv')])
for f in csv_files:
    path = os.path.join(merged_dir, f)
    size_mb = os.path.getsize(path) / (1024 * 1024)
    print(f"  {f}: {size_mb:.2f} MB")

# 2. Load and extract info from event timings
print("\n" + "=" * 60)
print("EVENT TIMINGS")
print("=" * 60)
df_events = pd.read_csv(os.path.join(merged_dir, 'merged_event_timings.csv'))
print(f"Total rows: {len(df_events)}")
print(f"Unique subjects: {df_events['subject'].nunique()}")
print(f"Unique events: {df_events['Event'].unique().tolist()}")
df_events['Duration'] = df_events['End_Time'] - df_events['Start_Time']
print("\nDuration (minutes) per event type:")
print(df_events.groupby('Event')['Duration'].agg(['mean', 'min', 'max']).round(2))
print("\nSample:")
print(df_events.head(10))

# 3. Load and extract info from questionnaire responses
print("\n" + "=" * 60)
print("QUESTIONNAIRE RESPONSES (PANAS, STAI, DIM, SSSQ)")
print("=" * 60)
df_quest = pd.read_csv(os.path.join(merged_dir, 'merged_questionnaire_responses.csv'))
print(f"Total rows: {len(df_quest)}")
print(f"Unique subjects: {df_quest['subject'].nunique()}")
print(f"Columns: {list(df_quest.columns)}")
print("\nResponses per subject:")
print(df_quest.groupby('subject').size())
print("\nDescriptive stats (numeric cols):")
numeric_cols = df_quest.select_dtypes(include=[np.number]).columns
print(df_quest[numeric_cols].describe().round(2))

# 4. Load and extract info from sensor CSVs (sample for large files)
print("\n" + "=" * 60)
print("SENSOR DATA SUMMARY")
print("=" * 60)
sensor_files = [f for f in csv_files if f.startswith('merged_') and f != 'merged_event_timings.csv' and f != 'merged_questionnaire_responses.csv']
for f in sensor_files:
    path = os.path.join(merged_dir, f)
    size_mb = os.path.getsize(path) / (1024 * 1024)
    # Use nrows for large files to avoid memory issues
    nrows = 100000 if size_mb > 50 else None
    df_sensor = pd.read_csv(path, nrows=nrows)
    rows_info = f"{len(df_sensor)} rows (sample)" if nrows else f"{len(df_sensor)} rows"
    print(f"\n{f}:")
    print(f"  Shape: {df_sensor.shape}, {rows_info}")
    print(f"  Columns: {list(df_sensor.columns)}")
    if 'subject' in df_sensor.columns:
        print(f"  Subjects: {df_sensor['subject'].unique().tolist()}")
    sensor_cols = [c for c in df_sensor.columns if c != 'subject']
    if sensor_cols:
        print(f"  Sample stats:\n{df_sensor[sensor_cols].describe().round(4)}")

EDA for merged datasets

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

merged_dir = '/Users/mostafazamaniturk/Documents/USD/AAI-530/work/WESAD_data/merged_cleaned_data'
df_events = pd.read_csv(os.path.join(merged_dir, 'merged_event_timings.csv'))
df_quest = pd.read_csv(os.path.join(merged_dir, 'merged_questionnaire_responses.csv'))
df_events['Duration'] = df_events['End_Time'] - df_events['Start_Time']

# --- 1. EVENT TIMINGS EDA ---
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Event duration distribution by event type
ax1 = axes[0]
event_order = ['Base', 'TSST', 'Medi 1', 'Fun', 'Medi 2', 'sRead', 'fRead', 'bRead']
df_plot = df_events[df_events['Event'].isin(event_order)]
sns.boxplot(data=df_plot, x='Event', y='Duration', order=event_order, ax=ax1, palette='viridis')
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, ha='right')
ax1.set_ylabel('Duration (minutes)')
ax1.set_title('Event Duration by Event Type')
ax1.grid(axis='y', alpha=0.3)

# Events per subject
ax2 = axes[1]
events_per_subject = df_events.groupby('subject')['Event'].count()
events_per_subject.plot(kind='bar', ax=ax2, color='steelblue', edgecolor='black', alpha=0.8)
ax2.set_xlabel('Subject')
ax2.set_ylabel('Number of Events')
ax2.set_title('Events Recorded per Subject')
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45, ha='right')
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# --- 2. QUESTIONNAIRE EDA ---
numeric_cols = df_quest.select_dtypes(include=[np.number]).columns
quest_numeric = df_quest[numeric_cols].drop(columns=[c for c in numeric_cols if 'SSSQ' in c], errors='ignore')

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# PANAS summary (first 10 items)
panas_cols = [c for c in quest_numeric.columns if c.startswith('PANAS_')][:10]
if panas_cols:
    panas_means = quest_numeric[panas_cols].mean()
    axes[0,0].barh(range(len(panas_means)), panas_means.values, color='coral', alpha=0.8)
    axes[0,0].set_yticks(range(len(panas_means)))
    axes[0,0].set_yticklabels(panas_cols, fontsize=8)
    axes[0,0].set_xlabel('Mean Score')
    axes[0,0].set_title('PANAS Mean Scores (first 10 items)')
    axes[0,0].grid(axis='x', alpha=0.3)

# STAI summary
stai_cols = [c for c in quest_numeric.columns if c.startswith('STAI_')]
if stai_cols:
    stai_means = quest_numeric[stai_cols].mean()
    axes[0,1].bar(range(len(stai_means)), stai_means.values, color='teal', alpha=0.8)
    axes[0,1].set_xticks(range(len(stai_means)))
    axes[0,1].set_xticklabels(stai_cols, rotation=45, ha='right')
    axes[0,1].set_ylabel('Mean Score')
    axes[0,1].set_title('STAI Mean Scores')
    axes[0,1].grid(axis='y', alpha=0.3)

# DIM summary
dim_cols = [c for c in quest_numeric.columns if c.startswith('DIM_')]
if dim_cols:
    axes[1,0].boxplot([quest_numeric[c].dropna() for c in dim_cols], labels=dim_cols)
    axes[1,0].set_xticklabels(dim_cols, rotation=45, ha='right')
    axes[1,0].set_ylabel('Score')
    axes[1,0].set_title('DIM Score Distribution')
    axes[1,0].grid(axis='y', alpha=0.3)

# Correlation heatmap (sample of questionnaire scales)
scale_cols = panas_cols[:5] + stai_cols + dim_cols if panas_cols else stai_cols + dim_cols
corr_cols = [c for c in scale_cols if c in quest_numeric.columns]
if len(corr_cols) >= 2:
    corr = quest_numeric[corr_cols].corr()
    sns.heatmap(corr, annot=True, fmt='.2f', cmap='RdBu_r', center=0, ax=axes[1,1], 
                xticklabels=True, yticklabels=True, annot_kws={'size': 7})
    axes[1,1].set_title('Questionnaire Scale Correlations')

plt.tight_layout()
plt.show()

In [ ]:
# --- 3. SENSOR DATA EDA (with sampling for large files) ---
sensor_files = [f for f in os.listdir(merged_dir) if f.startswith('merged_') 
                and f not in ['merged_event_timings.csv', 'merged_questionnaire_responses.csv']]

# Load sensors with sampling for files > 50 MB
sensor_dfs = {}
for f in sensor_files:
    path = os.path.join(merged_dir, f)
    size_mb = os.path.getsize(path) / (1024 * 1024)
    nrows = 50000 if size_mb > 50 else None  # Sample for large files
    sensor_dfs[f] = pd.read_csv(path, nrows=nrows)

In [ ]:
# Sensor distributions - univariate signals (EDA, BVP, ECG, EMG, Resp, Temp)
univariate_sensors = ['merged_chest_eda.csv', 'merged_wrist_eda.csv', 'merged_wrist_bvp.csv', 
                      'merged_chest_ecg.csv', 'merged_chest_emg.csv', 'merged_chest_resp.csv',
                      'merged_chest_temp.csv', 'merged_wrist_temp.csv']

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for i, fname in enumerate(univariate_sensors[:8]):
    if fname not in sensor_dfs:
        continue
    df = sensor_dfs[fname]
    sensor_col = [c for c in df.columns if c != 'subject'][0]
    ax = axes[i]
    df[sensor_col].hist(bins=50, ax=ax, color='steelblue', edgecolor='black', alpha=0.7)
    ax.set_title(f'{fname.replace("merged_", "").replace(".csv", "")}\n({sensor_col})')
    ax.set_xlabel(sensor_col)
    ax.set_ylabel('Count')
    ax.grid(alpha=0.3)

plt.suptitle('Sensor Value Distributions', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Subject-level sensor summaries (mean values per subject)
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Chest EDA by subject
df_eda = sensor_dfs.get('merged_chest_eda.csv')
if df_eda is not None and 'subject' in df_eda.columns:
    eda_by_subj = df_eda.groupby('subject')['EDA'].agg(['mean', 'std'])
    eda_by_subj['mean'].plot(kind='bar', ax=axes[0,0], color='darkgreen', alpha=0.8)
    axes[0,0].set_title('Mean Chest EDA by Subject')
    axes[0,0].set_ylabel('EDA (µS)')
    axes[0,0].tick_params(axis='x', rotation=45)
    axes[0,0].grid(axis='y', alpha=0.3)

# Wrist BVP by subject
df_bvp = sensor_dfs.get('merged_wrist_bvp.csv')
if df_bvp is not None and 'subject' in df_bvp.columns:
    bvp_by_subj = df_bvp.groupby('subject')['BVP'].mean()
    bvp_by_subj.plot(kind='bar', ax=axes[0,1], color='purple', alpha=0.8)
    axes[0,1].set_title('Mean Wrist BVP by Subject')
    axes[0,1].set_ylabel('BVP')
    axes[0,1].tick_params(axis='x', rotation=45)
    axes[0,1].grid(axis='y', alpha=0.3)

# Chest vs Wrist EDA comparison
df_wrist_eda = sensor_dfs.get('merged_wrist_eda.csv')
if df_eda is not None and df_wrist_eda is not None:
    chest_eda = df_eda.groupby('subject')['EDA'].mean()
    wrist_eda = df_wrist_eda.groupby('subject')['EDA'].mean()
    subjs = chest_eda.index.intersection(wrist_eda.index)
    x = np.arange(len(subjs))
    w = 0.35
    axes[1,0].bar(x - w/2, chest_eda.reindex(subjs).values, w, label='Chest EDA', color='darkgreen', alpha=0.8)
    axes[1,0].bar(x + w/2, wrist_eda.reindex(subjs).values, w, label='Wrist EDA', color='darkorange', alpha=0.8)
    axes[1,0].set_xticks(x)
    axes[1,0].set_xticklabels(subjs, rotation=45, ha='right')
    axes[1,0].set_ylabel('Mean EDA')
    axes[1,0].set_title('Chest vs Wrist EDA by Subject')
    axes[1,0].legend()
    axes[1,0].grid(axis='y', alpha=0.3)

# Accelerometer (wrist) - magnitude distribution
df_wrist_acc = sensor_dfs.get('merged_wrist_acc.csv')
if df_wrist_acc is not None:
    acc_cols = ['ACC_x', 'ACC_y', 'ACC_z']
    if all(c in df_wrist_acc.columns for c in acc_cols):
        df_wrist_acc = df_wrist_acc.copy()
        df_wrist_acc['ACC_mag'] = np.sqrt(df_wrist_acc['ACC_x']**2 + df_wrist_acc['ACC_y']**2 + df_wrist_acc['ACC_z']**2)
        df_wrist_acc['ACC_mag'].hist(bins=60, ax=axes[1,1], color='navy', alpha=0.7, edgecolor='black')
        axes[1,1].set_title('Wrist Accelerometer Magnitude')
        axes[1,1].set_xlabel('Magnitude')
        axes[1,1].set_ylabel('Count')
        axes[1,1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# --- 4. DATA QUALITY & MISSING VALUES ---
print("=" * 60)
print("MISSING VALUES & DATA QUALITY SUMMARY")
print("=" * 60)

# Event timings
print("\n1. Event Timings:")
print(f"   Missing Start_Time: {df_events['Start_Time'].isna().sum()}")
print(f"   Missing End_Time: {df_events['End_Time'].isna().sum()}")

# Questionnaire
print("\n2. Questionnaire Responses:")
print(df_quest.isnull().sum().to_string())

# Sensor data (sample)
print("\n3. Sensor Data - Missing Values (per sensor):")
for fname, df in sensor_dfs.items():
    missing = df.isnull().sum()
    if missing.any():
        print(f"   {fname}: {missing.to_dict()}")

In [ ]:
# --- 5. EDA KEY FINDINGS ---
print("=" * 60)
print("EDA KEY FINDINGS")
print("=" * 60)
print("""
1. EVENT TIMINGS
   - 15 subjects, 8 event types (Base, TSST, Medi 1, Medi 2, Fun, sRead, fRead, bRead)
   - Base: ~20 min, TSST: ~11 min, Medi/Fun: ~6-7 min, Read tasks: ~1.2 min
   - All subjects have consistent event structure

2. QUESTIONNAIRE
   - 75 rows (5 responses per subject for PANAS/STAI/DIM across conditions)
   - SSSQ has only 15 non-null rows (1 per subject)
   - Scales show expected ranges (1-5 for PANAS/STAI, 1-9 for DIM)

3. SENSOR DATA
   - Chest ACC: very large (~55M rows) - use sampling for analysis
   - Chest vs Wrist EDA: different scales (chest typically higher)
   - Chest Temp: possible outliers (min -273.15 suggests sensor errors)
   - Wrist ACC: integer values, typical range -128 to 127

4. DATA QUALITY
   - Check chest Temp for sensor malfunction (min -273.15°C)
   - SSSQ mostly missing except one response per subject
""")

## Extract Information from Merged CSVs

Extract structured metadata and summary statistics from each merged CSV file for downstream analysis and reporting.

In [ ]:
import os
import pandas as pd
import numpy as np

merged_dir = '/Users/mostafazamaniturk/Documents/USD/AAI-530/work/WESAD_data/merged_cleaned_data'
csv_files = sorted([f for f in os.listdir(merged_dir) if f.endswith('.csv')])
LARGE_FILE_THRESHOLD = 1_000_000  # Use sampling for files > 1M rows

extraction_results = {}

for fname in csv_files:
    fpath = os.path.join(merged_dir, fname)
    info = {'file': fname, 'path': fpath}
    
    # Load data (full or sampled for large files)
    if os.path.getsize(fpath) > 100_000_000:  # >100MB: use sample
        df = pd.read_csv(fpath, nrows=100_000)
        n_rows = len(df)  # Approximate; full count would require iterating
        info['n_rows'] = f">100k (file ~{os.path.getsize(fpath)/1e6:.0f} MB)"
        info['sampled'] = True
    else:
        df = pd.read_csv(fpath)
        n_rows = len(df)
        info['n_rows'] = n_rows
        info['sampled'] = n_rows > LARGE_FILE_THRESHOLD
    
    info['columns'] = list(df.columns)
    info['dtypes'] = {c: str(d) for c, d in df.dtypes.items()}
    
    # Numeric columns for statistics
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = [c for c in df.columns if c not in num_cols]
    
    if num_cols:
        stats = df[num_cols].describe().to_dict()
        info['stats'] = {col: {k: float(v) if np.isfinite(v) else None 
                               for k, v in s.items()} for col, s in stats.items()}
    
    if 'subject' in df.columns:
        info['subjects'] = sorted(df['subject'].unique().tolist())
        info['n_subjects'] = len(info['subjects'])
        # Per-subject row counts
        info['rows_per_subject'] = df.groupby('subject').size().to_dict()
    
    # Event timings: extract event info
    if fname == 'merged_event_timings.csv':
        info['events'] = df['Event'].unique().tolist() if 'Event' in df.columns else []
        info['event_durations'] = (df['End_Time'] - df['Start_Time']).describe().to_dict() if 'End_Time' in df.columns else {}
    
    # Questionnaire: extract scale summaries
    if fname == 'merged_questionnaire_responses.csv':
        panas_cols = [c for c in df.columns if c.startswith('PANAS_')]
        stai_cols = [c for c in df.columns if c.startswith('STAI_')]
        if panas_cols:
            info['panas_summary'] = df[panas_cols].describe().iloc[1:4].to_dict()
        if stai_cols:
            info['stai_summary'] = df[stai_cols].describe().iloc[1:4].to_dict()
    
    # Missing values
    info['missing_counts'] = df.isnull().sum().to_dict()
    info['missing_pct'] = (df.isnull().sum() / len(df) * 100).round(2).to_dict()
    
    extraction_results[fname] = info
    print(f"Extracted: {fname} ({n_rows:,} rows, {len(df.columns)} cols)")

# Summary DataFrame
summary_rows = []
for fname, info in extraction_results.items():
    summary_rows.append({
        'file': fname,
        'n_rows': info['n_rows'],
        'n_cols': len(info['columns']),
        'n_subjects': info.get('n_subjects', ''),
        'sampled': info.get('sampled', False),
    })
df_extraction_summary = pd.DataFrame(summary_rows)
print("\n--- EXTRACTION SUMMARY ---")
print(df_extraction_summary.to_string(index=False))

# Per-file extracted info available in extraction_results dict
# Example: extraction_results['merged_event_timings.csv']['events']

In [ ]:
# Display extracted information by category

# 1. Event timings
ev = extraction_results.get('merged_event_timings.csv', {})
if ev:
    print("EVENT TIMINGS:")
    print(f"  Events: {ev.get('events', [])}")
    print(f"  Subjects: {ev.get('n_subjects')}")
    if ev.get('event_durations'):
        dur = ev['event_durations']
        mn, me, mx = dur.get('min', 0), dur.get('mean', 0), dur.get('max', 0)
        print(f"  Duration (min): {float(mn):.2f}s, mean: {float(me):.2f}s, max: {float(mx):.2f}s")

# 2. Questionnaire
quest = extraction_results.get('merged_questionnaire_responses.csv', {})
if quest:
    print("\nQUESTIONNAIRE:")
    print(f"  Subjects: {quest.get('n_subjects')}")
    print(f"  Scale columns: PANAS (26), STAI (6), DIM (2), SSSQ (6)")

# 3. Sensor files - per-subject row counts
print("\nSENSOR DATA - ROWS PER SUBJECT (sample files):")
for fname in ['merged_wrist_eda.csv', 'merged_chest_eda.csv', 'merged_wrist_temp.csv']:
    info = extraction_results.get(fname, {})
    if info and 'rows_per_subject' in info:
        rps = info['rows_per_subject']
        print(f"  {fname}: min={min(rps.values()):,}, max={max(rps.values()):,}")

# 4. Stats sample for one sensor
print("\nSAMPLE STATS (merged_wrist_bvp.csv):")
bvp = extraction_results.get('merged_wrist_bvp.csv', {})
if bvp and 'stats' in bvp and 'BVP' in bvp.get('stats', {}):
    for k, v in bvp['stats']['BVP'].items():
        if v is not None:
            print(f"  BVP {k}: {v:.4f}")

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
print('Listing contents of MyDrive:')
! ls "/Users/mostafazamaniturk/Documents/USD/AAI-530/work/WESAD_data/merged_cleaned_data"

In [ ]:
merged_dir = '/Users/mostafazamaniturk/Documents/USD/AAI-530/work/WESAD_data/merged_cleaned_data'
bvp_dir = '/Users/mostafazamaniturk/Documents/USD/AAI-530/work/WESAD_data/merged_cleaned_data/merged_wrist_bvp.csv'
eda_dir = '/Users/mostafazamaniturk/Documents/USD/AAI-530/work/WESAD_data/merged_cleaned_data/merged_wrist_eda.csv'
event_timing_dir = '/Users/mostafazamaniturk/Documents/USD/AAI-530/work/WESAD_data/merged_cleaned_data/merged_event_timings.csv'

df_bvp = pd.read_csv(bvp_dir)
df_eda = pd.read_csv(eda_dir)
df_event_timing = pd.read_csv(event_timing_dir)


In [ ]:
df_bvp.columns

In [ ]:
df_bvp.head()


In [ ]:
df_bvp.tail()

In [ ]:
df_eda.columns

In [ ]:
df_event_timing.columns

In [ ]:
df_eda.head()

In [ ]:
import pandas as pd
import numpy as np
import sys

# ==========================================
# 1. CONFIGURATION (CHECK THESE!)
# ==========================================
FILE_BVP = bvp_dir
FILE_EDA = eda_dir
OUTPUT_FILE = 'aligned_wrist_data_64Hz.csv'

# Sampling Rates (Fixed by Empatica E4 hardware)
FS_BVP = 64.0  # Hz
FS_EDA = 4.0   # Hz

# ==========================================
# 2. LOAD DATA & DETECT COLUMNS
# ==========================================
print("--- Loading Data Previews ---")
# Load just the first row to check headers
bvp_preview = pd.read_csv(FILE_BVP, nrows=1)
eda_preview = pd.read_csv(FILE_EDA, nrows=1)

print(f"BVP Columns found: {list(bvp_preview.columns)}")
print(f"EDA Columns found: {list(eda_preview.columns)}")

# *** UPDATE THESE STRINGS BASED ON THE PRINT OUTPUT ABOVE ***
# Example: If your column is named 'subject_id', change 'id' to 'subject_id'
COL_SUBJECT = 'subject'   # The column identifying the person (e.g. S2, S3...)
COL_BVP_VAL = 'BVP'       # The column with Blood Volume Pulse values
COL_EDA_VAL = 'EDA'       # The column with Skin Conductance values

# Verify columns exist before proceeding
if COL_SUBJECT not in bvp_preview.columns:
    print(f"\n[ERROR] Column '{COL_SUBJECT}' not found in BVP file.")
    print(f"Please look at the 'BVP Columns found' above and update COL_SUBJECT in the script.")
    sys.exit()

# ==========================================
# 3. ALIGNMENT FUNCTION
# ==========================================
def align_subject(subject_id, df_bvp_sub, df_eda_sub):
    """
    Aligns low-freq EDA (4Hz) to high-freq BVP (64Hz) for a single subject.
    """
    # 1. Generate Relative Time Indices (Seconds since start)
    # -----------------------------------------------------
    # BVP Time: 0, 0.015625, 0.03125...
    n_bvp = len(df_bvp_sub)
    time_bvp = np.arange(n_bvp) / FS_BVP
    
    # EDA Time: 0, 0.25, 0.50...
    n_eda = len(df_eda_sub)
    time_eda = np.arange(n_eda) / FS_EDA

    # 2. Create Series with Time as Index
    # -----------------------------------------------------
    bvp_series = pd.Series(df_bvp_sub[COL_BVP_VAL].values, index=time_bvp)
    eda_series = pd.Series(df_eda_sub[COL_EDA_VAL].values, index=time_eda)

    # 3. Upsample EDA to match BVP Time
    # -----------------------------------------------------
    # Reindex creates NaNs at the new 64Hz timestamps
    # Interpolate fills them linearly
    # ffill/bfill handles the very first/last fractional seconds
    eda_aligned = eda_series.reindex(bvp_series.index).interpolate(method='linear').ffill().bfill()

    # 4. Combine into a clean DataFrame
    # -----------------------------------------------------
    df_aligned = pd.DataFrame({
        COL_SUBJECT: subject_id,
        'time_sec': bvp_series.index,  # Relative time (0.00 to end)
        'bvp': bvp_series.values,
        'eda': eda_aligned.values
    })
    
    return df_aligned

# ==========================================
# 4. MAIN PROCESSING LOOP
# ==========================================
print("\n--- Loading Full Datasets ---")
df_bvp_all = pd.read_csv(FILE_BVP)
df_eda_all = pd.read_csv(FILE_EDA)

print(f"BVP Total Rows: {len(df_bvp_all)}")
print(f"EDA Total Rows: {len(df_eda_all)}")

aligned_dfs = []
unique_subjects = df_bvp_all[COL_SUBJECT].unique()
print(f"\nProcessing {len(unique_subjects)} subjects: {unique_subjects}")

for sub_id in unique_subjects:
    # Extract this subject's data
    sub_bvp = df_bvp_all[df_bvp_all[COL_SUBJECT] == sub_id]
    sub_eda = df_eda_all[df_eda_all[COL_SUBJECT] == sub_id]

    # Safety check: Does this subject exist in both files?
    if sub_eda.empty:
        print(f"  [WARN] Subject {sub_id} found in BVP but MISSING in EDA. Skipping.")
        continue

    # Run Alignment
    try:
        aligned_chunk = align_subject(sub_id, sub_bvp, sub_eda)
        aligned_dfs.append(aligned_chunk)
    except Exception as e:
        print(f"  [ERROR] Failed to align Subject {sub_id}: {e}")

# ==========================================
# 5. SAVE RESULTS
# ==========================================
if aligned_dfs:
    print("\n--- Merging & Saving ---")
    final_df = pd.concat(aligned_dfs, ignore_index=True)
    
    final_df.to_csv(OUTPUT_FILE, index=False)
    print(f"Success! Aligned data saved to: {OUTPUT_FILE}")
    print(f"Final Shape: {final_df.shape}")
    print(final_df.head())
else:
    print("\n[ERROR] No data was aligned. Check your inputs.")

In [ ]:
# Add labels from merged_event_timings to aligned_wrist_data_64Hz.csv
# Uses merged_event_timings.csv - event Start_Time/End_Time are in MINUTES (from quest); convert to seconds.
# Label: 0=Baseline, 1=Stress(TSST), 2=Amusement(Fun), -1=Other/Transition(Medi,Read)

import pandas as pd
import numpy as np
import os

ALIGNED_FILE = '/Users/mostafazamaniturk/Documents/USD/AAI-530/work/edaFolder/aligned_wrist_data_64Hz.csv'
EVENT_TIMINGS_FILE = '/Users/mostafazamaniturk/Documents/USD/AAI-530/work/WESAD_data/merged_cleaned_data/merged_event_timings.csv'

EVENT_TO_LABEL = {'Base': 0, 'TSST': 1, 'Fun': 2}  # Medi, Read, etc -> -1

df = pd.read_csv(ALIGNED_FILE)
df_events = pd.read_csv(EVENT_TIMINGS_FILE)
df_events['label'] = df_events['Event'].map(lambda e: EVENT_TO_LABEL.get(e, -1))

# Event times in merged_event_timings are in MINUTES from session start; time_sec is in seconds.
labels = np.full(len(df), -1, dtype=np.int64)
for subj in df['subject'].unique():
    mask = (df['subject'] == subj).values
    idx = np.where(mask)[0]
    times = df.loc[mask, 'time_sec'].values
    subj_events = df_events[df_events['subject'] == subj]
    for _, ev in subj_events.iterrows():
        start_sec = ev['Start_Time'] * 60.0
        end_sec = ev['End_Time'] * 60.0
        in_range = (times >= start_sec) & (times <= end_sec)
        labels[idx] = np.where(in_range, ev['label'], labels[idx])

df['label'] = labels

# Save labeled dataset
df.to_csv(ALIGNED_FILE, index=False)
print(f'Done! Labels added. Saved to {ALIGNED_FILE}')
print(f'Label distribution:\n{df["label"].value_counts().sort_index()}')
print(f'\nSample (first 10 rows):')
print(df.head(10))

In [ ]:
print('Listing contents of MyDrive:')
! ls "/Users/mostafazamaniturk/Documents/USD/AAI-530/work/edaFolder"

In [ ]:
aligned_dir = '/Users/mostafazamaniturk/Documents/USD/AAI-530/work/edaFolder/aligned_wrist_data_64Hz.csv'

df = pd.read_csv(aligned_dir)


In [ ]:
df.columns

In [ ]:
df.head()


In [ ]:
df.tail()


In [ ]:
df.describe()


In [ ]:
df.info()


In [ ]:
df.isnull().sum()

In [ ]:
# Verify labeled data (run after the label-assignment cell above)
ALIGNED_FILE = '/Users/mostafazamaniturk/Documents/USD/AAI-530/work/edaFolder/aligned_wrist_data_64Hz.csv'
df = pd.read_csv(ALIGNED_FILE)
print('Label distribution (0=baseline, 1=stress, 2=amusement, -1=other):')
print(df['label'].value_counts().sort_index())
print('\nColumns:', list(df.columns))
print('\nSample with labels:')
print(df.head(10))



### Alternative: Full Pipeline from Original Pickle (Guarantees Full Session)

If merged CSVs were created with truncated data, run this cell to load directly from pickle files, align wrist BVP+EDA, and assign labels using `merged_event_timings.csv`. Ensures entire session (~1.5 hr/subject) is exported.

In [ ]:
# Full pipeline from pickle: load, align, label. Run if merged CSVs may be truncated.
import os
import pandas as pd
import numpy as np

base_path = '/Users/mostafazamaniturk/Documents/USD/AAI-530/work'
wesad_base = os.path.join(base_path, 'WESAD_data', 'WESAD')
merged_dir = os.path.join(base_path, 'WESAD_data', 'merged_cleaned_data')
SUBJECTS = ['S2','S3','S4','S5','S6','S7','S8','S9','S10','S11','S13','S14','S15','S16','S17']
FS_BVP, FS_EDA = 64.0, 4.0
EVENT_TO_LABEL = {'Base': 0, 'TSST': 1, 'Fun': 2}

def _clean_split(s):
    return [p.strip() for p in s.replace('#','').split(';') if p.strip()]

aligned_dfs = []
for subj in SUBJECTS:
    pkl_path = os.path.join(wesad_base, subj, f'{subj}.pkl')
    quest_path = os.path.join(wesad_base, subj, f'{subj}_quest.csv')
    if not os.path.exists(pkl_path) or not os.path.exists(quest_path):
        print(f"Skipping {subj}: missing files"); continue
    raw = pd.read_pickle(pkl_path)
    bvp = raw['signal']['wrist']['BVP'].flatten()
    eda = raw['signal']['wrist']['EDA'].flatten()
    # Align EDA (4Hz) to BVP (64Hz)
    time_bvp = np.arange(len(bvp)) / FS_BVP
    time_eda = np.arange(len(eda)) / FS_EDA
    eda_s = pd.Series(eda, index=time_eda).reindex(time_bvp).interpolate().ffill().bfill()
    df = pd.DataFrame({'subject':subj, 'time_sec':time_bvp, 'bvp':bvp, 'eda':eda_s.values})
    # Event timings from quest (Start/End in minutes)
    q = pd.read_csv(quest_path)
    order = _clean_split(q.iloc[0,0])[1:]
    parts_s = _clean_split(q.iloc[1,0])
    parts_e = _clean_split(q.iloc[2,0])
    labels = np.full(len(df), -1, dtype=np.int64)
    for i, e in enumerate(order):
        if i+1 >= len(parts_s) or i+1 >= len(parts_e): continue
        try:
            st_min, end_min = float(parts_s[i+1]), float(parts_e[i+1])
            lbl = EVENT_TO_LABEL.get(e, -1)
            in_range = (df['time_sec'] >= st_min*60) & (df['time_sec'] <= end_min*60)
            labels = np.where(in_range, lbl, labels)
        except (ValueError, IndexError): pass
    df['label'] = labels
    aligned_dfs.append(df)
    print(f"  {subj}: {len(df)} rows, {len(df)/FS_BVP/60:.1f} min")

if aligned_dfs:
    out = pd.concat(aligned_dfs, ignore_index=True)
    out_path = os.path.join(base_path, 'edaFolder', 'aligned_wrist_data_64Hz.csv')
    out.to_csv(out_path, index=False)
    print(f"\nSaved {out_path} | Shape: {out.shape}")
    print(f"Label counts:\n{out['label'].value_counts().sort_index()}")

---
## Part 2: Exploratory Data Analysis (Aligned Wrist 64Hz)
---

In [ ]:
# Path to aligned wrist BVP/EDA data for EDA section
DATA_PATH = '/Users/mostafazamaniturk/Documents/USD/AAI-530/work/aligned_wrist_data_64Hz_v1.csv'
if not os.path.exists(DATA_PATH):
    DATA_PATH = os.path.join(os.getcwd(), 'aligned_wrist_data_64Hz.csv')

## 1. Load Data & Basic Info

In [ ]:
df = pd.read_csv(DATA_PATH)
df.head(10)

In [ ]:
df.tail(10)

### Class Imbalance Check (Stress Prediction)

For ML stress prediction: exclude label -1 (other/transition), use labels 0=baseline, 1=stress, 2=amusement.  
Stress (1) is the target of interest—check imbalance and get class weights for training.

In [ ]:
label_names = {-1: 'other', 0: 'baseline', 1: 'stress', 2: 'amusement'}

# ML-ready subset: exclude -1 (other/transition); use only 0, 1, 2 for stress prediction
df_ml = df[df['label'] >= 0].copy()
print(f"ML dataset (labels 0,1,2 only): {len(df_ml):,} samples\n")

# --- 3-class: baseline, stress, amusement ---
counts_3 = df_ml['label'].value_counts().sort_index()
total_3 = len(df_ml)
stress_count = counts_3.get(1, 0)
stress_pct = 100 * stress_count / total_3

print("=" * 55)
print("STRESS PREDICTION — CLASS IMBALANCE CHECK")
print("=" * 55)
print("\n--- 3-Class (baseline, stress, amusement) ---")
for lbl in [0, 1, 2]:
    cnt = counts_3.get(lbl, 0)
    pct = 100 * cnt / total_3
    marker = " ★ TARGET" if lbl == 1 else ""
    print(f"  {label_names[lbl]} ({lbl}): {cnt:,} samples ({pct:.2f}%){marker}")

majority_3 = counts_3.idxmax()
minority_3 = counts_3.idxmin()
ratio_3 = counts_3.max() / counts_3.min()
print(f"\n  Imbalance ratio: {ratio_3:.2f}  |  Stress samples: {stress_count:,}")

# --- Binary: stress (1) vs non-stress (0+2) ---
df_binary = df_ml.copy()
df_binary['stress_binary'] = (df_binary['label'] == 1).astype(int)
counts_bin = df_binary['stress_binary'].value_counts()
stress_bin = counts_bin.get(1, 0)
non_stress_bin = counts_bin.get(0, 0)
ratio_bin = max(stress_bin, non_stress_bin) / min(stress_bin, non_stress_bin)

print("\n--- Binary (stress vs non-stress) ---")
print(f"  stress (1):    {stress_bin:,} samples ({100*stress_bin/total_3:.2f}%)")
print(f"  non-stress:    {non_stress_bin:,} samples ({100*non_stress_bin/total_3:.2f}%)")
print(f"  Imbalance ratio: {ratio_bin:.2f}")

# --- Class weights for sklearn (3-class) ---
try:
    from sklearn.utils.class_weight import compute_class_weight
    y_3 = df_ml['label'].values
    weights_3 = compute_class_weight('balanced', classes=np.array([0, 1, 2]), y=y_3)
    class_weights_3 = {i: w for i, w in zip([0, 1, 2], weights_3)}
except ImportError:
    # Fallback: balanced weight = n_samples / (n_classes * n_samples_per_class)
    n = len(df_ml)
    class_weights_3 = {lbl: n / (3 * cnt) for lbl, cnt in counts_3.items()}
print("\n--- Class weights (for 3-class ML) ---")
for lbl in sorted(class_weights_3):
    print(f"  {label_names[lbl]} ({lbl}): {class_weights_3[lbl]:.4f}")
print("  Use in model: class_weight=class_weights_3")

# --- Verdict & recommendations ---
print("\n--- Verdict ---")
if ratio_3 > 1.5:
    print(f"⚠️  Dataset is IMBALANCED (ratio {ratio_3:.2f} > 1.5)")
    print("\n  ML recommendations for stress prediction:")
    print("  • Use class_weight='balanced' or class_weights_3 above")
    print("  • Consider SMOTE/oversampling on stress class to boost recall")
    print("  • Use recall/F1 for stress class—avoid missing stressed users")
    print("  • Stratify train/test splits: stratify=df_ml['label']")
else:
    print(f"✓ Dataset is relatively balanced (ratio {ratio_3:.2f})")

# Keep df_ml for model training
print(f"\ndf_ml ready: {len(df_ml):,} samples for stress prediction")

In [ ]:
df.tail(10)

In [ ]:
print('Shape:', df.shape)
print('\nColumns:', df.columns.tolist())
print('\nData types:')
print(df.dtypes)
print('\nMemory usage:', df.memory_usage(deep=True).sum() / 1024**2, 'MB')

## 2. Label Distribution

WESAD labels: **0**=baseline, **1**=stress, **2**=amusement, **-1**=other/transition (Medi, Read events).

In [ ]:
# Label distribution
label_counts = df['label'].value_counts().sort_index()
label_names = {-1: 'other', 0: 'baseline', 1: 'stress', 2: 'amusement'}
print('Label distribution:')
for k, v in label_counts.items():
    print(f'  {label_names.get(k, k)} ({k}): {v:,} ({100*v/len(df):.1f}%)')
print(f'\nThree-class (0,1,2) samples: {(df["label"] >= 0).sum():,}')

fig, ax = plt.subplots(figsize=(8, 4))
labels_sorted = sorted(label_counts.index)
colors = ['gray', 'steelblue', 'coral', 'seagreen']
ax.bar([label_names.get(l, str(l)) for l in labels_sorted], 
       [label_counts[l] for l in labels_sorted], 
       color=[colors[i] for i in range(len(labels_sorted))], edgecolor='black')
ax.set_xlabel('Label')
ax.set_ylabel('Count')
ax.set_title('Distribution of Stress/Affect Labels')
plt.tight_layout()
plt.show()

### BVP and EDA by Label

Compare signal statistics across stress conditions.

In [ ]:
# Filter to three-class labels (exclude -1) for cleaner comparison
df_labels = df[df['label'] >= 0].copy()
df_labels['label_name'] = df_labels['label'].map(label_names)

# Per-label statistics
agg_label = df_labels.groupby('label').agg({'bvp': ['mean', 'std'], 'eda': ['mean', 'std']})
print('BVP and EDA statistics by label:')
print(agg_label.round(4))

# Boxplots by label (order: baseline, stress, amusement)
order = ['baseline', 'stress', 'amusement']
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
for ax, col in zip(axes, ['bvp', 'eda']):
    data_by_label = [df_labels[df_labels['label'] == k][col] for k in [0, 1, 2]]
    bp = ax.boxplot(data_by_label, labels=order, patch_artist=True)
    for patch in bp['boxes']:
        patch.set_facecolor('lightsteelblue')
    ax.set_title(f'{col.upper()} by Label')
    ax.set_ylabel(col.upper())
plt.tight_layout()
plt.show()

## 2. Missing Values

In [ ]:
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
pd.DataFrame({'Missing Count': missing, 'Missing %': missing_pct})

## 3. Statistical Summary

In [ ]:
df.describe(include='all')

## 4. Subject Distribution

In [ ]:
subject_counts = df['subject'].value_counts().sort_index()
print('Number of unique subjects:', df['subject'].nunique())
print('\nRows per subject:')
print(subject_counts)
print('\nDuration per subject (approx minutes, at 64 Hz):')
for subj in subject_counts.index:
    n = subject_counts[subj]
    dur_min = n / 64 / 60
    print(f'  {subj}: {dur_min:.1f} min ({n:,} samples)')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
subject_counts.plot(kind='bar', ax=ax, color='steelblue', edgecolor='black')
ax.set_title('Sample Count per Subject')
ax.set_xlabel('Subject')
ax.set_ylabel('Number of Samples')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

## 5. Univariate Distributions

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# BVP histogram
axes[0, 0].hist(df['bvp'].dropna(), bins=80, color='coral', edgecolor='black', alpha=0.7)
axes[0, 0].set_title('BVP (Blood Volume Pulse) Distribution')
axes[0, 0].set_xlabel('BVP value')

# EDA histogram
axes[0, 1].hist(df['eda'].dropna(), bins=80, color='seagreen', edgecolor='black', alpha=0.7)
axes[0, 1].set_title('EDA (Electrodermal Activity) Distribution')
axes[0, 1].set_xlabel('EDA value (µS)')

# Time distribution
axes[1, 0].hist(df['time_sec'], bins=50, color='slateblue', edgecolor='black', alpha=0.7)
axes[1, 0].set_title('Time Distribution (seconds)')
axes[1, 0].set_xlabel('time_sec')

# Box plots for BVP and EDA
df[['bvp', 'eda']].boxplot(ax=axes[1, 1])
axes[1, 1].set_title('BVP vs EDA Box Plots')

plt.tight_layout()
plt.show()

## 6. Time Series Overview

In [ ]:
# Sample one subject for visualization (first 5000 points ~78 sec)
sample_subj = df['subject'].iloc[0]
sample = df[df['subject'] == sample_subj].head(5000)

fig, axes = plt.subplots(2, 1, figsize=(14, 6), sharex=True)
axes[0].plot(sample['time_sec'], sample['bvp'], color='coral', linewidth=0.5)
axes[0].set_ylabel('BVP')
axes[0].set_title(f'BVP Time Series - Subject {sample_subj} (first ~78 sec)')
axes[0].grid(True, alpha=0.3)

axes[1].plot(sample['time_sec'], sample['eda'], color='seagreen', linewidth=0.5)
axes[1].set_ylabel('EDA (µS)')
axes[1].set_xlabel('Time (seconds)')
axes[1].set_title(f'EDA Time Series - Subject {sample_subj}')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Time Series Colored by Label

BVP and EDA for one subject with segments colored by condition.

In [ ]:
# Time series with label-colored segments (sample ~3 min of one subject)
sample_subj = df['subject'].iloc[0]
sample = df[df['subject'] == sample_subj].head(12000)  # ~3 min at 64 Hz

label_colors = {-1: 'lightgray', 0: 'steelblue', 1: 'coral', 2: 'seagreen'}
fig, axes = plt.subplots(2, 1, figsize=(14, 6), sharex=True)

for ax, col, ylab in zip(axes, ['bvp', 'eda'], ['BVP', 'EDA (µS)']):
    for lbl in [-1, 0, 1, 2]:
        mask = sample['label'] == lbl
        if mask.any():
            ax.scatter(sample.loc[mask, 'time_sec'], sample.loc[mask, col], 
                      c=label_colors[lbl], s=1, alpha=0.6, label=label_names.get(lbl, lbl))
    ax.set_ylabel(ylab)
    ax.legend(loc='upper right', markerscale=5)
    ax.grid(True, alpha=0.3)
axes[1].set_xlabel('Time (seconds)')
axes[0].set_title(f'BVP & EDA by Label - Subject {sample_subj}')
plt.tight_layout()
plt.show()

## 7. Correlation Analysis

In [ ]:
corr = df[['bvp', 'eda']].corr()
print('Correlation matrix:')
print(corr)

# Scatter plot (sample for readability)
sample_size = min(20000, len(df))
sample_df = df.sample(n=sample_size, random_state=42)

fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(sample_df['bvp'], sample_df['eda'], alpha=0.3, s=5)
ax.set_xlabel('BVP')
ax.set_ylabel('EDA (µS)')
ax.set_title('BVP vs EDA Scatter (sampled 20k points)')
plt.tight_layout()
plt.show()

## 8. Per-Subject Statistics

In [ ]:
agg = df.groupby('subject').agg({
    'bvp': ['mean', 'std', 'min', 'max', 'median'],
    'eda': ['mean', 'std', 'min', 'max', 'median']
}).round(4)
agg

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

df.boxplot(column='bvp', by='subject', ax=axes[0])
axes[0].set_title('BVP by Subject')
axes[0].set_xlabel('Subject')

df.boxplot(column='eda', by='subject', ax=axes[1])
axes[1].set_title('EDA by Subject')
axes[1].set_xlabel('Subject')

plt.suptitle('')
plt.tight_layout()
plt.show()

## 9. Outlier Detection

In [ ]:
def iqr_outliers(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return (series < lower) | (series > upper)

bvp_out = iqr_outliers(df['bvp'])
eda_out = iqr_outliers(df['eda'])

print('BVP outliers (IQR method):', bvp_out.sum(), f'({bvp_out.sum()/len(df)*100:.2f}%)')
print('EDA outliers (IQR method):', eda_out.sum(), f'({eda_out.sum()/len(df)*100:.2f}%)')
print('\nBVP outlier bounds:', df['bvp'].quantile(0.25) - 1.5*(df['bvp'].quantile(0.75)-df['bvp'].quantile(0.25)), 'to',
      df['bvp'].quantile(0.75) + 1.5*(df['bvp'].quantile(0.75)-df['bvp'].quantile(0.25)))
print('EDA outlier bounds:', df['eda'].quantile(0.25) - 1.5*(df['eda'].quantile(0.75)-df['eda'].quantile(0.25)), 'to',
      df['eda'].quantile(0.75) + 1.5*(df['eda'].quantile(0.75)-df['eda'].quantile(0.25)))

## 10. Sampling Rate Verification

In [ ]:
# Verify 64 Hz: delta should be ~1/64 sec
sample = df[df['subject'] == df['subject'].iloc[0]].head(100)
deltas = sample['time_sec'].diff().dropna()
print('Expected sampling interval: 1/64 =', 1/64, 'sec')
print('Observed mean interval:', deltas.mean(), 'sec')
print('Observed std of intervals:', deltas.std(), 'sec')
print('Implied sampling rate:', 1/deltas.mean(), 'Hz')